CCDR Hazard Analysis Notebook
Developed by M. Amadio and T. Iwanaga

In [1]:
from common import *  # import necessary packages

%matplotlib inline

In [2]:
country_code_map = {
    "NPL": 175,
    "PAK": 188,
    "BGD": 23,
    "GHA": 94,
    "ETH": 79,
    "BFA": 42,
    "MLI": 155,
    "NER": 181,
    "TCD": 50,
    "MRT": 159
}

In [3]:
def run_analysis(rb):
    with output:
        output.clear_output()
        print("Running analysis...")
        rb.disabled = True

    # Ensure values are valid
    bin_seq = [w.value for w in class_edges.values()]

    seq = np.all([True if bin_seq[i] < bin_seq[i+1] else False for i in range(0, len(bin_seq)-1)])

    if not seq:
        ValueError("Class thresholds are not sequential. Lower classes must be less than class thresholds above.")
        rb.disabled = False
        return

    max_bin_value = np.max(bin_seq)
    max_haz_threshold = max_bin_value + 1e-4
    bin_seq = bin_seq + [np.inf]
    num_bins = len(bin_seq)

    # Get user input
    country = country_dd.value
    exp_cat = exp_cat_dd.value
    target_ADM = adm_dd.value
    adm_name = target_ADM.replace('_', '')

    min_haz_threshold = class_edges['class_1'].value

    valid_RPs = [5, 20, 100]

    # Testing data file locations
    # TODO: Temp data store, to be replaced with a config spec (.env file?) before deployment
        
    if exp_cat_dd.value == 'pop':
        try:
            exp_ras = f"{DATA_DIR}/EXP/{country}_WPOP20.tif"
        except ValueError:
            # TODO: Download from website if layer not available
            exp_ras = f"{DATA_DIR}/EXP/{country}_WPOP20.tif"
    elif exp_cat_dd.value == 'builtup':
        exp_ras = f"{DATA_DIR}/EXP/{country}_WSF19.tif"
    elif exp_cat_dd.value == 'agri':
        exp_ras = f"{DATA_DIR}/EXP/{country}_ESA20_agri.tif"
    else:
        ValueError("Missing data layer")

    # Hazard data location
    hazard_RP_data_loc = f"{DATA_DIR}/HZD"

    # Run analysis

    # Open exposure dataset
    exp_data = rxr.open_rasterio(exp_ras)

    # Indicate -1 values as representing no data.
    exp_data.rio.write_nodata(-1, inplace=True)

    # Load ADM based on country code value
    try:
        adm_dataset = gpd.read_file(os.path.join(DATA_DIR, f"ADM/{country}_ADM.gpkg"), layer=f"{country}_{adm_name}")
    except ValueError:
        print("Missing ADM layer!")

    adm_data = adm_dataset.loc[adm_dataset.ADM0_CODE == country_code_map[country], :]
    
    # Get all ADM code/name columns to save with results
    adm_cols = adm_data.columns
    all_adm_codes = adm_data.columns.str.contains("_CODE")
    all_adm_names = adm_data.columns.str.contains("_NAME")
    
    all_adm_name_tmp = adm_cols[all_adm_names].tolist()
    all_adm_code_tmp = adm_cols[all_adm_codes].to_list()

    result_df = adm_data.loc[:, all_adm_code_tmp + all_adm_name_tmp + ["geometry"]]
    
    # Prep result structure     
    for rp in valid_RPs:
        
        # Get total exposure for each ADM region
        exp_per_ADM = gen_zonal_stats(vectors=adm_data["geometry"], raster=exp_ras, stats=["sum"])
        
        result_df[f"{adm_name}_{exp_cat}"] = [x['sum'] for x in exp_per_ADM]

        # Load corresponding hazard dataset
        hazard_rst = rxr.open_rasterio(os.path.join(hazard_RP_data_loc, f"GLB_HS_RP{rp}.tif"))

        # Reproject and clip raster to same bounds as exposure data
        hazard_rst = hazard_rst.rio.reproject_match(exp_data)

        # Get raw array values for exposure and hazard layer
        hazard_arr = hazard_rst[0].values

        hazard_arr[hazard_arr < min_haz_threshold] = 0  # Set values below min threshold to nan
        hazard_arr[hazard_arr > max_bin_value] = max_haz_threshold  # Cap large values to maximum threshold value
        
        # Calculate affected exposure in ADM        
        # Filter down to valid areas
        valid_impact_areas = hazard_rst.values > 0
        affected_exp = exp_data.where(valid_impact_areas)  # Get total exposure in affected areas
        affected_exp = affected_exp.where(affected_exp > 0)  # Out of the above, get areas that have people
        
        if save_inter_rst_chk.value:
            affected_exp.rio.to_raster(os.path.join(OUTPUT_DIR, f"{country}_HS_{rp}_{exp_cat}_hazard_affected_.tif"))
            
        # Conduct analyses for classes

        # Assign bin values to raster data
        # Follows: x_{i-1} <= x_{i} < x_{i+1}
        bin_idx = np.digitize(hazard_arr, bin_seq)

        for bin_x in range(1, num_bins):
            impact_class = gen_zonal_stats(vectors=adm_data["geometry"], raster=np.array(bin_idx == bin_x).astype(int) * affected_exp.data[0],
                                stats=["sum"], affine=affected_exp.rio.transform(), nodata=np.nan)
            result_df[f"RP{rp}_{exp_cat}_C{bin_x}"] = [x['sum'] for x in impact_class]
        # end

        C_cols = result_df.columns.str.contains(f"RP{rp}_{exp_cat}_C")
        result_df[f"RP{rp}_{exp_cat}_tot_exposed"] = result_df.loc[:, C_cols].sum(axis=1)

    # End RP loop
    
    # Round to three decimal places to avoid giving the impression of high precision
    result_df = result_df.round(3)

    # Write table of total population in each class, in each ADM2
    df_cols = result_df.columns

    no_geom = result_df.loc[:, df_cols[~df_cols.isin(['geometry'])]].fillna(0)
    no_geom.to_csv(os.path.join(OUTPUT_DIR, f"{country}_HS_{adm_name}_{exp_cat}_class.csv"), index=False)
    result_df.to_file(os.path.join(OUTPUT_DIR, f"{country}_HS_{adm_name}_{exp_cat}_class.gpkg"))

    with output:
        print("Finished analysis.")
        rb.disabled = False


In [4]:
# Data option widgets
country_dd = widgets.Dropdown(
    options=[('Nepal', 'NPL'), ('Pakistan', 'PAK'),('Bangladesh', 'BGD'),('Ghana', 'GHA'),
            ('Ethiopia', 'ETH'), ('Burkina Faso', 'BFA'), ('Mali', 'MLI'), ('Niger', 'NER'),
            ('Chad', 'TCD'), ('Mauritania', 'MRT')],
    value='NPL',
    description='Country:',
    style={'description_width': 'initial'}
)

exp_cat_dd = widgets.Dropdown(
    options=[("Population", "pop"), ("Built-up", "builtup"), ("Agriculture", "agri")],
    value='pop',
    description='Exposure Category:',
    style={'description_width': 'initial'}
)

adm_dd = widgets.Dropdown(
    options=['ADM1', 'ADM2', 'ADM3'],
    value='ADM2',
    description='Administrative Unit Level:',
    style={'description_width': 'initial'}
)

analysis_app_dd = widgets.Dropdown(
    options=["Classes"],
    value="Classes",
    description='Analysis Approach:',
    style={'description_width': 'initial'}
)

# Class value inputs
class_edges = OrderedDict({
    f'class_{i+1}': widgets.BoundedFloatText(
        value=k,
        min=18.0,
        max=50.0,
        step=1.0,
        description=f'Class {i+1}:',
        tooltip=f'Minimum value of class {i+1}. Value must be less than the next entry.',
        disabled=False
    ) for (i,k) in enumerate([18, 23, 28, 30])
})

# User action widgets
save_inter_rst_chk = widgets.Checkbox(
    value=False,
    description='Export Intermediate Rasters',
    tooltip='Save rasters generated between each step (saves to nominated output directory)',
    disabled=False,
    indent=False
)

# Run button to perform analysis
run_button = widgets.Button(
    description='Run Analysis',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to run analysis with selected options',
    # icon='check' # (FontAwesome names without the `fa-` prefix)
)

reset_display_button = widgets.Button(
    description='Reset',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reset display',
    # icon='check' # (FontAwesome names without the `fa-` prefix)
)

def reset_display(bt):
    output.clear_output()
    run_button.disabled = False

run_button.on_click(run_analysis)
reset_display_button.on_click(reset_display)

def analysis_change(change):
    if change.new == "Classes":
        disable_class = False
        
    for w in class_edges.values():
        w.disabled = disable_class

analysis_app_dd.observe(analysis_change, names='value')

In [5]:
display(country_dd)
display(exp_cat_dd)
display(adm_dd)
display(analysis_app_dd)
[display(w) for w in class_edges.values()]

display(HBox([run_button, save_inter_rst_chk]), 
        reset_display_button)

output = widgets.Output()
display(output)

Dropdown(description='Country:', options=(('Nepal', 'NPL'), ('Pakistan', 'PAK'), ('Bangladesh', 'BGD'), ('Ghan…

Dropdown(description='Exposure Category:', options=(('Population', 'pop'), ('Built-up', 'builtup'), ('Agricult…

Dropdown(description='Administrative Unit Level:', index=1, options=('ADM1', 'ADM2', 'ADM3'), style=Descriptio…

Dropdown(description='Analysis Approach:', options=('Classes',), style=DescriptionStyle(description_width='ini…

BoundedFloatText(value=18.0, description='Class 1:', max=50.0, min=18.0, step=1.0)

BoundedFloatText(value=23.0, description='Class 2:', max=50.0, min=18.0, step=1.0)

BoundedFloatText(value=28.0, description='Class 3:', max=50.0, min=18.0, step=1.0)

BoundedFloatText(value=30.0, description='Class 4:', max=50.0, min=18.0, step=1.0)

Button(description='Reset', style=ButtonStyle(), tooltip='Reset display')

Output()